<a href="https://colab.research.google.com/github/Minhyuk03/KMOU_2-2_ML/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_%EA%B3%BC%EC%A0%9C_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필요한 라이브러리 임포트
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import os
from PIL import Image, ImageEnhance

# 2. 이미지 데이터 경로 및 기본 설정
BASE_DIR = '/content/drive/MyDrive/test_image/'
CLASSES = ['cat', 'dog']
IMG_SIZE = (128, 128)
USE_COLOR = True
USE_AUGMENTATION = True

# 3. 데이터 증강 함수 정의
def augment_image(img):
    """이미지 증강: 밝기, 대비, 좌우 반전"""
    augmented_images = [img]

    # 좌우 반전
    augmented_images.append(img.transpose(Image.FLIP_LEFT_RIGHT))

    # 밝기 조정
    enhancer = ImageEnhance.Brightness(img)
    augmented_images.append(enhancer.enhance(1.2))
    augmented_images.append(enhancer.enhance(0.8))

    # 대비 조정
    enhancer = ImageEnhance.Contrast(img)
    augmented_images.append(enhancer.enhance(1.3))

    return augmented_images

# 4. 데이터 로드 및 전처리 함수 정의
def load_and_preprocess_data(base_dir, classes, img_size, use_color=True, use_augmentation=False):
    data_X = []
    data_Y = []

    label_map = {cls: i for i, cls in enumerate(classes)}
    file_list = os.listdir(base_dir)

    for filename in file_list:
        class_name = None

        if filename.startswith('cat.') and filename.endswith('.jpg'):
            class_name = 'cat'
        elif filename.startswith('dog.') and filename.endswith('.jpg'):
            class_name = 'dog'
        else:
            continue

        file_path = os.path.join(base_dir, filename)

        if use_color:
            img = Image.open(file_path).convert('RGB')
        else:
            img = Image.open(file_path).convert('L')

        img = img.resize(img_size)

        if use_augmentation:
            images_to_process = augment_image(img)
        else:
            images_to_process = [img]

        for processed_img in images_to_process:
            img_array = np.array(processed_img).flatten()
            img_array = img_array / 255.0

            data_X.append(img_array)
            data_Y.append(label_map[class_name])

    X = np.array(data_X)
    Y = np.array(data_Y)

    return X, Y

# 5. 데이터 로드 및 분할 실행
X, Y = load_and_preprocess_data(BASE_DIR, CLASSES, IMG_SIZE,
                                 use_color=USE_COLOR,
                                 use_augmentation=USE_AUGMENTATION)

if len(X) == 0:
    print("\n[실행 중단] 로드된 이미지 데이터가 없습니다. 경로와 파일명을 확인해 주세요.")
else:
    print(f"총 {len(X)}개의 이미지 로드 완료.")
    print(f"데이터 크기 (X): {X.shape}, 레이블 크기 (Y): {Y.shape}")
    print(f"학습 데이터 개수: {int(len(X)*0.8)}, 테스트 데이터 개수: {int(len(X)*0.2)}")

    # 학습 데이터와 테스트 데이터 분할
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y,
        test_size=0.2,
        random_state=42,
        stratify=Y
    )

    # 특징 스케일링
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 7. SVM 모델 학습 및 평가
    print("\n")
    print("SVM 모델 학습 및 성능 분석")

    svm_model = SVC(kernel='rbf', C=10.0, gamma='scale', random_state=42)
    svm_model.fit(X_train_scaled, Y_train)
    Y_pred_svm = svm_model.predict(X_test_scaled)

    conf_matrix_svm = confusion_matrix(Y_test, Y_pred_svm)
    print("SVM 혼동 행렬 (Confusion Matrix):")
    print(f"   [Cat(0) 예측] [Dog(1) 예측]")
    print(f"Cat(0) 실제: {conf_matrix_svm[0]}")
    print(f"Dog(1) 실제: {conf_matrix_svm[1]}")

    print(f"정확도 (Accuracy): {accuracy_score(Y_test, Y_pred_svm):.4f}")
    print(f"정밀도 (Precision): {precision_score(Y_test, Y_pred_svm, zero_division=0):.4f}")
    print(f"재현율 (Recall): {recall_score(Y_test, Y_pred_svm, zero_division=0):.4f}")
    print(f"F1-Score: {f1_score(Y_test, Y_pred_svm, zero_division=0):.4f}")

총 2000개의 이미지 로드 완료.
데이터 크기 (X): (2000, 49152), 레이블 크기 (Y): (2000,)
학습 데이터 개수: 1600, 테스트 데이터 개수: 400


SVM 모델 학습 및 성능 분석
SVM 혼동 행렬 (Confusion Matrix):
   [Cat(0) 예측] [Dog(1) 예측]
Cat(0) 실제: [182  18]
Dog(1) 실제: [ 20 180]
정확도 (Accuracy): 0.9050
정밀도 (Precision): 0.9091
재현율 (Recall): 0.9000
F1-Score: 0.9045
